## 智能路由

本节，为了实现 **智能路由**，我们来写前文提到的 **客诉核查 Agent**。

它的设定是这样的。我们是一家电商公司，现在有一位客户向我们发起了客诉，我们需要一个 Agent，自动验证客诉的内容是否属实。

这个 Agent 本质是一个决策体，它的功能是根据预先设定好的客诉类型，将客诉转入对应的客诉流程。我们希望待决策问题不要过于简单，起码是 `if else` 无法实现的，否则我们的 Agent 就会变成画蛇添足的产物。

### 1. 构造样本数据

第一步是喜闻乐见的编数据环节。我们要编两张 PostgreSQL 表：订单表、物流表。

下面是建表语句：

```sql
-- 创建数据库
CREATE DATABASE ecommerce_orders;

-- 创建新用户
CREATE USER admin WITH ENCRYPTED PASSWORD 'admin-password';

-- 授予用户权限
GRANT ALL PRIVILEGES ON DATABASE ecommerce_orders TO admin;

-- 切换数据库
\c ecommerce_orders

-- 订单表
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    uid INTEGER NOT NULL,
    mall_id INTEGER NOT NULL,
    goods_id INTEGER NOT NULL,
    status VARCHAR(20) NOT NULL,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT valid_status CHECK (status IN ('ordered', 'cancelled'))
);

-- 物流表
CREATE TABLE logistics (
    order_id INTEGER PRIMARY KEY,
    status VARCHAR(20) NOT NULL,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT valid_status CHECK (status IN ('pending', 'in_transit', 'delivered', 'cancelled'))
);

-- 赋予表权限
GRANT SELECT ON orders TO admin;
GRANT SELECT ON logistics TO admin;
```

上面是两张无历史记录的状态表。实际业务中，历史记录一般存在 Hive 表，PostgreSQL 表存当前状态就好了。

为了让 Agent 理解如何使用这两张表，还需要为它添加注释。


```sql
-- 订单表注释
COMMENT ON TABLE orders IS '用户订单信息';

-- 订单表字段注释
COMMENT ON COLUMN orders.order_id IS '唯一订单ID（主键）';
COMMENT ON COLUMN orders.uid IS '用户ID';
COMMENT ON COLUMN orders.mall_id IS '商城ID';
COMMENT ON COLUMN orders.goods_id IS '商品ID';
COMMENT ON COLUMN orders.status IS '订单状态: ordered(已下单)/cancelled(已取消)';
COMMENT ON COLUMN orders.timestamp IS '订单状态更新时间';

-- 物流表注释
COMMENT ON TABLE logistics IS '订单的物流状态信息';

-- 物流表字段注释
COMMENT ON COLUMN logistics.order_id IS '关联的订单ID（主键';
COMMENT ON COLUMN logistics.status IS '物流状态: pending(待处理)/in_transit(运输中)/delivered(已送达)/cancelled(已取消)';
COMMENT ON COLUMN logistics.timestamp IS '物流状态更新时间';
```

让 DeepSeek 帮我造一些订单：

```sql
-- 插入订单数据
INSERT INTO orders (order_id, uid, mall_id, goods_id, status, timestamp) VALUES
(1001, 101, 1, 5001, 'ordered', '2025-05-01 10:00:00'),  -- 正常下单待发货
(1002, 102, 2, 6002, 'ordered', '2025-05-02 14:30:00'),  -- 运输中订单
(1003, 103, 1, 5003, 'ordered', '2025-05-03 09:15:00'),  -- 已送达订单
(1004, 104, 3, 7004, 'cancelled', '2025-05-04 16:45:00'), -- 发货前取消
(1005, 105, 2, 6005, 'cancelled', '2025-05-05 11:20:00'); -- 运输中取消

-- 插入物流数据
INSERT INTO logistics (order_id, status, timestamp) VALUES
(1001, 'pending', '2025-05-01 10:05:00'),     -- 待发货状态
(1002, 'in_transit', '2025-05-02 15:00:00'),   -- 运输中状态
(1003, 'delivered', '2025-05-03 17:30:00'),    -- 已送达状态
(1004, 'cancelled', '2025-05-04 16:50:00'),    -- 发货前取消
(1005, 'in_transit', '2025-05-05 11:30:00');   -- 取消时已在运输中
```

> PostgreSQL 数据库的安装过程见 [postgresql_bot.ipynb](test_qwen_agent/3.postgresql_bot.ipynb)

### 2. Python 连接 PostgreSQL

检查能否获取 PostgreSQL 中的数据。

In [1]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="ecommerce_orders",
    user="admin",
    password="admin-password"
)

In [2]:
cursor = conn.cursor()
cursor.execute("SELECT version();")
record = cursor.fetchone()
record

('PostgreSQL 16.9 (Ubuntu 16.9-0ubuntu0.24.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0, 64-bit',)

In [3]:
with conn.cursor() as cursor:
    cursor.execute("SELECT * FROM orders;")

    # 获取所有结果
    records = cursor.fetchall()

    # 输出查询结果
    for row in records:
        print(row)

(1001, 101, 1, 5001, 'ordered', datetime.datetime(2025, 5, 1, 10, 0))
(1002, 102, 2, 6002, 'ordered', datetime.datetime(2025, 5, 2, 14, 30))
(1003, 103, 1, 5003, 'ordered', datetime.datetime(2025, 5, 3, 9, 15))
(1004, 104, 3, 7004, 'cancelled', datetime.datetime(2025, 5, 4, 16, 45))
(1005, 105, 2, 6005, 'cancelled', datetime.datetime(2025, 5, 5, 11, 20))


In [4]:
with conn.cursor() as cursor:
    cursor.execute("SELECT * FROM logistics;")

    # 获取所有结果
    records = cursor.fetchall()

    # 输出查询结果
    for row in records:
        print(row)

(1001, 'pending', datetime.datetime(2025, 5, 1, 10, 5))
(1002, 'in_transit', datetime.datetime(2025, 5, 2, 15, 0))
(1003, 'delivered', datetime.datetime(2025, 5, 3, 17, 30))
(1004, 'cancelled', datetime.datetime(2025, 5, 4, 16, 50))
(1005, 'in_transit', datetime.datetime(2025, 5, 5, 11, 30))


In [5]:
if conn:
    cursor.close()
    conn.close()
    print("数据库连接已关闭")

数据库连接已关闭


### 3. MCP 调用 PostgreSQL

首先启动 vLLM 服务，来到项目根目录：

```bash
cd test_openai_agent
bash vllm_server.sh
```